In [27]:
import numpy as np
from scipy.integrate import RK45

In [140]:
# Simulation constants
n_timesteps = 10000
dt = 0.01
n_pedestrians = 1

In [161]:
# Pedestrian contants
relaxation_times = np.ones(n_pedestrians)[:,None] * 0.5
anisotropic_character = 0.75
sigma = 0
radii = np.ones(n_pedestrians) * 0.3
maximal_speeds = desired_speeds[0] * 1.3
A1, A2, B1, B2 = (0, 2, 0.3, 0.2)
destinations = np.array([[0, 20] for i in range(n_pedestrians)])

In [162]:
# Simulation state
positions = np.empty((n_timesteps, n_pedestrians, 2,), dtype=float) # [timesteps, pedestrians, (x,y)]
velocities = np.empty((n_timesteps, n_pedestrians, 2,), dtype=float)
desired_speeds = np.empty((n_timesteps, n_pedestrians, 1), dtype=float)

positions[0] = np.array([[i,(-1)**i] for i in range(n_pedestrians)])
desired_speeds[0] = 1 #1.34
velocities[0] = np.array([0,1])

In [100]:
def driving_force(desired_directions, desired_speeds, velocities):
        return 1/relaxation_times * (desired_speeds * desired_directions - velocities)

In [119]:
def calc_radii_sum(radii):
    return np.add.outer(radii, radii)

def calc_rel_pos(positions):
    return positions[:, None, :] - positions[None, :, :]

def calc_dists(relative_positions):
    distances = np.linalg.norm(relative_positions, axis=2)
    np.fill_diagonal(distances, np.inf)
    return distances

def calc_norm_vecs(relative_positions, distances):
    return relative_positions /  distances[:, :, None]

def calc_fovs(velocities, norm_vectors):
    velocity_norms = velocities / np.linalg.norm(velocities, axis=1, keepdims=True)
    cos_phi = -np.sum(norm_vectors * velocity_norms[:, None, :], axis=2, keepdims=True)
    return anisotropic_character + (1-anisotropic_character) * (1+cos_phi) / 2

def repulsive_force(positions, radii, velocities):
    radii_sum = calc_radii_sum(radii)
    relative_positions = calc_rel_pos(positions)
    distances = calc_dists(relative_positions)
    norm_vectors = calc_norm_vecs(relative_positions, distances)

    fov = calc_fovs(velocities, norm_vectors)

    repulsive_force_mat = A1 * np.exp( (radii_sum - distances)/B1 )[:,:,None] \
                        * norm_vectors * fov \
                        + A2 * np.exp( (radii_sum - distances)/B2 )[:,:,None] \
                        * norm_vectors
    return np.sum(repulsive_force_mat, axis=1)

In [85]:
def boundary_force(positions):
    return np.zeros((n_pedestrians,2,))

In [51]:
def individuality_force(desired_directions):
    e_perp = np.zeros_like(desired_directions)
    e_perp[:,0] = -desired_directions[:,1]
    e_perp[:,1] = desired_directions[:,0]
    return rng.normal(0, sigma, size=n_pedestrians)[:, None]*e_perp

In [166]:
def total_force(positions, destinations, radii, desired_speeds, velocities):
    desired_directions = destinations - positions
    desired_directions /= np.linalg.norm(desired_directions, axis=1, keepdims=True)
    total_force = driving_force(desired_directions, desired_speeds, velocities) \
                + repulsive_force(positions, radii, velocities) \
                + boundary_force(positions) \
                + individuality_force(desired_directions)
    print("-------------------------------------")
    print(f"driving_force == {driving_force(desired_directions, desired_speeds, velocities)}")
    print(f"repulsive_force == {repulsive_force(positions, radii, velocities)}")
    print(f"boundary_force == {boundary_force(positions)}")
    print(f"individuality_force == {individuality_force(desired_directions)}")
    print("-------------------------------------")

    return total_force

In [155]:
def calc_desired_speeds(t, positions, desired_speeds, destinations):
    pos0_to_dest = destinations - positions[0]
    pos0_to_dest /= np.linalg.norm(pos0_to_dest, axis=1, keepdims=True)
    pos0_to_pos = positions - positions[0]
    progress_to_dest = np.einsum("nd,nd->n", pos0_to_pos, pos0_to_dest)[:,None]
    average_speeds = progress_to_dest * (0 if t == 0 else 1 / t)
    impatience = 1 - average_speeds / desired_speeds
    return (1-impatience) * desired_speeds[0] + impatience * maximal_speeds

In [54]:
def test_driving_force():
    desired_directions = np.array([[1,1], [2,1], [1,2]])
    desired_speeds = np.array([[1], [1], [1]])
    velocities = np.array([[1,1], [1,2], [2,1]])
    assert driving_force(desired_directions, desired_speeds, velocities).shape == (3,2)

In [180]:
rng.normal(0, 0, size=4)[:, None]

array([[0.],
       [0.],
       [0.],
       [0.]])

In [55]:
def test_calc_radii_sum():
    shape = (3,3)
    radii = np.array([.3,.3,.3])
    radii_sum_true = np.ones((3,3)) * 0.6
    radii_sum = calc_radii_sum(radii)
    assert shape == radii_sum.shape
    assert (radii_sum_true == radii_sum).all()

def test_calc_rel_pos():
    shape = (3,3,2)
    positions = np.array([[-1,0], [2,1], [1,3]])
    rel_pos_true = np.array([[[0, 0], [-3, -1], [-2, -3]],
                             [[3, 1], [0, 0], [1, -2]],
                             [[2, 3], [-1, 2], [0, 0]]])
    rel_pos = calc_rel_pos(positions)
    assert shape == rel_pos.shape
    assert (rel_pos_true == rel_pos).all()

def test_calc_dists():
    shape = (3,3)
    positions = np.array([[-1,0], [2,1], [1,3]])
    distances_true = np.array([[np.inf, np.sqrt(10), np.sqrt(13)],
                          [np.sqrt(10), np.inf, np.sqrt(5)],
                          [np.sqrt(13), np.sqrt(5), np.inf]])
    relative_positions = calc_rel_pos(positions)
    distances = calc_dists(relative_positions)
    assert shape == distances.shape
    assert (distances_true == distances).all()

def test_calc_norm_vecs():
    shape = (3,3,2)
    positions = np.array([[-1,0], [2,1], [1,3]])
    norm_vecs_true = np.array([[[0,0], [-3/np.sqrt(10), -1/np.sqrt(10)], [-2/np.sqrt(13), -3/np.sqrt(13)]],
                               [[3/np.sqrt(10), 1/np.sqrt(10)], [0,0], [1/np.sqrt(5), -2/np.sqrt(5)]],
                               [[2/np.sqrt(13), 3/np.sqrt(13)], [-1/np.sqrt(5), 2/np.sqrt(5)], [0,0]]])
    relative_positions = calc_rel_pos(positions)
    norm_vecs = calc_norm_vecs(relative_positions, calc_dists(relative_positions))
    assert shape == norm_vecs.shape
    assert (norm_vecs_true == norm_vecs).all()

def test_calc_fovs():
    shape = (3,3,1)
    positions = np.array([[-1,0], [2,1], [1,3]])
    velocities = np.array([[2,0], [1,-1], [1,1]])
    velocity_norms = np.array([[np.sqrt(2),0], [1,-1], [1,1]]) / np.sqrt(2)
    #cos_phi = np.array([[[0], [-3/np.sqrt(5)], [-2/np.sqrt(13/2)]],
    #                    [[2/np.sqrt(10)], [0], [-3/np.sqrt(5)]],
    #                    [[5/np.sqrt(13)], [1/np.sqrt(5)], [0]]])
    #fov_true = 0.75 + (1-0.75) * (1 + cos_phi) / 2
    relative_positions = calc_rel_pos(positions)
    fov = calc_fovs(velocities, calc_norm_vecs(relative_positions, calc_dists(relative_positions)))
    # Check shape
    assert fov.shape == (3,3,1)
    # Check values in expected range
    assert np.all(fov >= anisotropic_character)
    assert np.all(fov <= 1.0)

In [56]:
test_calc_radii_sum()
test_calc_rel_pos()
test_calc_dists()
test_calc_norm_vecs()
test_calc_fovs()

In [170]:
# Other
rng = np.random.default_rng(42)
### temporary
t0 = 0
y0 = np.concatenate((positions[0].flatten(),
                     velocities[0].flatten(),
                     desired_speeds[0].flatten()))
def RK45_step(t, y):
    positions_step = y[:2*n_pedestrians].reshape(positions[0].shape)
    velocities_step = y[2*n_pedestrians:4*n_pedestrians].reshape(velocities[0].shape)
    desired_speeds_step = y[4*n_pedestrians:].reshape(desired_speeds[0].shape)

    new_desired_speeds = calc_desired_speeds(t, positions_step, desired_speeds_step, destinations)
    dpositions = velocities_step
    dvelocities = total_force(positions_step, destinations, radii, desired_speeds_step, velocities_step)
    print("-------------------------------------")
    print(f"positions = {positions_step}")
    print(f"dpositions = {dpositions}")
    print(f"dvelocities = {dvelocities}")
    print("-------------------------------------")
    return np.concatenate((dpositions.flatten(),
                           dvelocities.flatten(),
                           new_desired_speeds.flatten()))
    
### /temporary
class_RK45 = RK45(RK45_step, t0, y0, n_timesteps*dt)

-------------------------------------
driving_force == [[0. 0.]]
repulsive_force == [[0. 0.]]
boundary_force == [[0. 0.]]
individuality_force == [[-0.  0.]]
-------------------------------------
-------------------------------------
positions = [[0. 1.]]
dpositions = [[0. 1.]]
dvelocities = [[0. 0.]]
-------------------------------------
-------------------------------------
driving_force == [[0.        0.0274573]]
repulsive_force == [[0. 0.]]
boundary_force == [[0. 0.]]
individuality_force == [[-0.  0.]]
-------------------------------------
-------------------------------------
positions = [[0.        1.0105605]]
dpositions = [[0. 1.]]
dvelocities = [[0.        0.0274573]]
-------------------------------------


In [172]:
for i in range(10):
    class_RK45.step()

-------------------------------------
driving_force == [[0.         0.05046526]]
repulsive_force == [[0. 0.]]
boundary_force == [[0. 0.]]
individuality_force == [[-0.  0.]]
-------------------------------------
-------------------------------------
positions = [[0.         1.01940971]]
dpositions = [[0. 1.]]
dvelocities = [[0.         0.05046526]]
-------------------------------------
-------------------------------------
driving_force == [[0.         0.07349397]]
repulsive_force == [[0. 0.]]
boundary_force == [[0. 0.]]
individuality_force == [[-0.  0.]]
-------------------------------------
-------------------------------------
positions = [[0.         1.02911457]]
dpositions = [[0.         1.00110196]]
dvelocities = [[0.         0.07349397]]
-------------------------------------
-------------------------------------
driving_force == [[0.         0.18770974]]
repulsive_force == [[0. 0.]]
boundary_force == [[0. 0.]]
individuality_force == [[-0.  0.]]
-----------------------------------

In [173]:
class_RK45.y

array([ 0.        , 19.7495864 ,  0.        , -3.00895643,  8.75032287])

In [178]:
a = 1
b = 1
a == b != None

True